In [ ]:
include("../../src/Yggdrasil.jl")
using .Yggdrasil

# Example description
This notebook has examples on how to plot data. 

The examples shown are to plot a single variable in a plot. 

The plotting routines here are based on a base 2D implementation. 
When called with higher dimensional data the function will first try to collapse dimension to check if lower-dimension implementation can be called. 
If not, then information about the higher dimension must be specified. 
Specifically, it is assumed that the two higher dimensions are "varibles" and "time" 

The possible calling sequennces are:

- plot_heatmap(data_2d)                 
- plot_heatmap(data_3d, var)           - where var is the index of the variable dimension      
- plot_heatmap(data_3d)                - here the 3D data must be collapseble to 2D - e.i only 1 variable
- plot_heatmap(data_4d, var, t)        - where var is the index of the varialbe dimension, and t is the index of the time dimension
- plot_heatmap(data_4d), var           - here the 4D data must be collapable to 3D - e.i. only 1 time step
- plot_heatmap(data_4d)                - here the 4D data must be collapsable to 2D - e.i. only 1 time step and 1 variable

In [ ]:
data_folder = "../example_data/test_01/"

#-------------- get meta information on initial snapshot --------------
snap = 101                                        # Number of the snapshot
initial_snap = read_snapshot(data_folder, snap)   # Load initial snapshot meta
IDX = initial_snap.IDX                            # Variables index information
system = initial_snap.SYSTEM;                     # Unit system
#-----------------------------------------------------------------------------

#--------------- define which snapshot to load ---------------
i_start = 101   # Start snapshot index
i_end  = 110    # End snapshot index
stride = 1      # Stride - how many snapshots to skip between loads
n_snaps = Int(floor( (i_end - i_start) / stride) + 1)  # total number of snapshots to load
load_pic = false; # If true, will only load PIC patch data

#------------ Get area defined by LLC and URC aligned with patches --------------   
    #------------ set refinement level -------------------
    ref_level = 3  # zero meants no refiment (coarsest grid level)
    #-----------------------------------------------------
    #----------- set level based on refinement level -----------
    level_min = initial_snap.LEVELMIN
    level = level_min + ref_level 
    #-----------------------------------------------------
    #------------- define midpoint of area and number of patches
    midpoint = [16.27207,  0.0, 6.1138]
    patches = 4
    #-----------------------------------------------------
llc, urc, llc_pos, urc_pos  = get_area(initial_snap, midpoint, patches, level);


#---------------------- load snapshots in the area ----------------------
#--------- first get cartesian coordinates for the area -----------
drop_dims = true  # drops array dimension with size 1 in the cartesian coordinates
    #--------- in example 2D case, y dimension is collapse, so we can ignore it like this 
x,_,z,ds = get_xyz(initial_snap, true, level; llc=llc, urc=urc);
#-------------------------------------------------------------------
    # optional get 1D coordinates for x and z
x_vals = x[:, 1]
z_vals = z[1, :];
#-------------------------------------------------------------------

#---------------- load snapshot data in the area ----------------------
data, times = load_multiple_snapshots(data_folder, 
                                      i_start, i_end, stride, 
                                      llc, urc,
                                      load_pic; 
                                      use_level=level);

In [ ]:
#-------------- simplest way to plot a 2D heatmap --------------
plot_heatmap(data[:,1,:,1,1])

In [ ]:
#--------------- example with ALL parameters listed ---------------
#         this is mainly just to show available parameters
plot_heatmap(data[:,1,:,1,1];                   # the 2D data to plot
    x = x_vals,                                 # x-coordinates (default = 1:nx)
    y = z_vals,                                 # y-coordinates (default = 1:ny)
    cmap = "plasma",                            # colormap
    xlabel = "x",                               # x-axis label
    ylabel = "y",                               # y-axis label
    label_fontsize = 16,                        # axis label font size
    tick_fontsize = 14,                         # tick label font size
    text_color = "black",                       # label/title colour
    tick_color = "black",                       # tick text colour
    show_xticks = true,                         # show x-axis ticks
    show_yticks = true,                         # show y-axis ticks
    show_xlabel = true,                         # show x-axis label
    show_ylabel = true,                         # show y-axis label
    xlim = nothing,                             # no x-axis limit override
    ylim = nothing,                             # no y-axis limit override
    title = nothing,                            # no title by default
    title_fontsize = 18,                        # title font size
    show_grid = false,                          # no grid by default
    grid_alpha = 0.3,                           # grid transparency
    grid_linewidth = 1.0,                       # grid line width
    show_colorbar = true,                       # include colourbar
    colorbar_title = "Field",                   # colourbar label
    cb_label_fontsize = 16,                     # colourbar label font size
    cb_tick_fontsize = 14,                      # colourbar tick font size
    show_colorbar_ticks = true,                 # show colourbar ticks
    show_colorbar_label = true,                 # show colourbar label
    colorbar_orientation = "vertical",          # vertical colourbar
    colorbar_pad = 0.02,                        # spacing to colourbar
    colorbar_fraction = 0.046,                  # relative size of colourbar
    colorbar_shrink = 0.98,                     # shrink colourbar
    colorbar_aspect = nothing,                  # default aspect
    colorbar_lim = nothing,                     # auto scale limits
    colorbar_nticks = nothing,                  # auto tick spacing
    overlay_lines = [],                         # no overlay lines by default
    figsize = (8, 6),                           # figure size
    dpi = 300,                                  # resolution
    bbox_inches = "tight",                      # crop padding
    save = false,                               # don't save to file
    figname = "field_plot.png",                 # default filename
    transparent = false,                        # no transparency
    background_color = nothing,                 # no background colour override
    return_objects = false                      # display instead of returning
)

In [ ]:
#-------------- function can also be called higher dimension arrays --------------
#          example with 3D data
#          index of the variable to plot must be specified
#          in which dimension the variable is located can be specified with optional var_idx
#                           the default is 3 (x,y,var), but can be changed to 1 (var,x,y) or 2 (x,var,y)
#
#         NOTE - var can be neglected if the only 1 varible is in the 3D data (e.i. the data has collapseble dimension)
var = 1 
data_3d = data[:,1,:,:,1]  # 3D data to plot

plot_heatmap(data_3d,
        var;
        x = x_vals,
        y = z_vals,
        var_idx = 3
)

In [ ]:
#-------------- function can also be called higher dimension arrays --------------
#          example with 4D data
#          index of the variable to plot must be specified
#          time index t must be specified
#          in which dimension the variables and time dimension are located can be specified with optional var_idx and time_idx
#                        the default is var_idx=3 and time_idx=4  corresponding to (x,y,var,t)
#
#          Note - var and t can be neglected if 4d data is collapseble (e.i. if only 1 variable or 1 time step)
#                 t may also just be neglected if the data has only 1 time step but still more varialbes (e.i) it is 3D
var = 1 
t   = 10
data_4d = drop_unit_dims(data)

plot_heatmap(data_4d,
        var,
        t;
        x = x_vals,
        y = z_vals,
        var_idx = 3,
        time_idx = 4
)

In [ ]:
û     = [0.707, 0.0, -0.68]
num_points = 50
half_length = 0.3
line = get_line(midpoint,
                  û,
                  half_length,
                  num_points);

p1 = [line[1][1], line[1][3]]
p2 = [line[end][1], line[end][3]]
overlay_lines = [(p1,p2, "black", "-", 2.0)]

In [ ]:
plot_heatmap(data_4d,
        var,
        t;
        x = x_vals,
        y = z_vals,
        overlay_lines = overlay_lines
)

In [ ]:
#---------------- return figure for further processing ----------------
#        this can be used if you want to adjust the figure beoynd the current implementations
fig, ax, im, cb =  plot_heatmap(data[:,1,:,1,1];
            return_objects = true)
display(fig)